# Physics 256
## Lecture 07 - Modules, filesystem & I/O

<img src="http://upload.wikimedia.org/wikipedia/commons/d/d4/Pi_pie2.jpg" width=300px>

## Last Time

- functions
- introduction to modules

## Today
- more modules
- input and output


### Writing modules

Any code or collection of methods we write can be included in a module by writing them in a text editor (i.e. not in the iPython notebook or interpreter) and saving them with a unqiue filename `mymodule.py`.  We can treat these as complete python programs that can be run, or we can import them from other scripts to maximize effeciency and build collections of useful libraries.

Modules and scripts can both named `mymodule.py` and both can contain variables, functions/methods and classes.

** What's the difference?**

* When imported, a module's `__name__` attribute is set to the module's file name, without *.py*.
* When executed as a script, the `__name__` attribute is set to `__main__`.
* Except for special cases, you shouldn't put any major executable code at the top-level. 
* Put code in functions, classes, methods, and guard it with if `__name__ == "__main__"`.

In [ ]:
%load data/mymodule.py

In [1]:
# then from another script (or from the iPython notebook) 
# we can import the functions defined in this script

# we need to tell python where the script lives, via the sys module
import sys
sys.path.append('data')

import mymodule

mymodule.foo()

1

## Computing $\pi$

Let's put these ideas into practice by writing a module containing functions that compute the value of $\pi$ in various ways.

### Gottfried Wilhelm Leibniz 1646-1716
<img src="http://upload.wikimedia.org/wikipedia/commons/6/6a/Gottfried_Wilhelm_von_Leibniz.jpg" width=200px>

\begin{equation}
\pi = 4 \sum_{n=1}^\infty \frac{(-1)^{n+1}}{2n-1}
\end{equation}

Abraham Sharp 1653-1742
<img src="http://upload.wikimedia.org/wikipedia/commons/4/43/Sharp_Abraham.jpg" width=200px>
\begin{equation}
\pi = \sum_{n=0}^\infty \frac{2(-1)^{n}3^{1/2-n}}{2n+1}
\end{equation}

Monte Carlo Calculation using `random.random()`
<img src="http://upload.wikimedia.org/wikipedia/commons/thumb/c/ce/Circle_Area.svg/265px-Circle_Area.svg.png" width="200px">
<img src="http://upload.wikimedia.org/wikipedia/commons/8/84/Pi_30K.gif" width="200px">


In [ ]:
%load data/pi.py

In [3]:
%time %run data/pi.py

Leibniz     π = 3.14159285358982
Sharp       π = 3.14159265358979
Monte Carlo π = 3.14182320000000
CPU times: user 6.47 s, sys: 7.85 ms, total: 6.48 s
Wall time: 6.49 s


In [4]:
%time import pi

Leibniz     π = 3.14159285358982
Sharp       π = 3.14159265358979
Monte Carlo π = 3.14087840000000
CPU times: user 6.4 s, sys: 8.4 ms, total: 6.41 s
Wall time: 6.42 s


In [5]:
import imp
%time imp.reload(pi)

CPU times: user 1.31 ms, sys: 1.02 ms, total: 2.33 ms
Wall time: 1.84 ms


<module 'pi' from 'data/pi.py'>

Let's compare the accuracy of our various methods

In [6]:
N = [10**n for n in range(3,7)]

# Get the Leibniz, Sharp and Monte Carlo approximations to pi
π = [(pi.leibniz(cN),pi.sharp(cN),pi.monte_carlo(cN)) for cN in N]

# Output the results to the terminal
print('%16s%16s%16s%16s' % ('N','Leibniz  π','Sharp  π','Monte Carlo π'))
for i,cN in enumerate(N):
    print('%16d%16.8f%16.8f%16.8f' % (cN,π[i][0],π[i][1],π[i][2]))

               N      Leibniz  π        Sharp  π   Monte Carlo π
            1000      3.14259365      3.14159265      3.10800000
           10000      3.14169266      3.14159265      3.13800000
          100000      3.14160265      3.14159265      3.13908000
         1000000      3.14159365      3.14159265      3.14316800


## Accessing the Filesytem

It can be very useful to use python to work with the filesystem.  This is most conveniently done with the use of two standard library modules:

In [7]:
import os
import shutil

### List all files in a directory

In [10]:
list_path = 'data'
for f in os.listdir(list_path):
    print(f)

.pi.py.swp
__pycache__
function.png
in.dat
leapyear.py
lists.png
mymodule.py
out.dat
pi.py
primes.py
sho_energy.dat


In [13]:
# what if we only want teh files (and not directories?)
for f in os.listdir(list_path):
    if os.path.isfile(f):
        print(f)

### Creating a directory

In [16]:
example_num = 2
dir_name = 'examples' + os.path.sep + 'example%02d' % example_num 
if not os.path.exists(dir_name):
    os.makedirs(dir_name)

### Copy (or move) files

In [17]:
for f in os.listdir(list_path):
    extension = os.path.splitext(f)[-1]
    if extension == '.py':
        dest = dir_name + os.path.sep + f
        src = list_path + os.path.sep + f
        shutil.copy(src,dest)  

### Can even create archive files (see the `shutil` help docs)

In [18]:
shutil.make_archive('example_archive', 'zip', '.', 'examples')

'/Users/adelmaes/Documents/UVM/Teaching/Phys256/Lectures/Notebooks/example_archive.zip'

<div class="span alert alert-success">
<h2> Team programming challenge </h2> <br />

Write a python script that produces the appropriately named .zip archive for your assignment
solutions.

You will need to:
<ul>
<li>specify a source directory where your solutions are located</li>
<li>specify the assignment number</li>
<li>create the target directories: Lastname_I/A0X/</li>
<li>copy the solution files into the target directory</li>
<li>create the .zip file</li>
<li>perform any required clean up </li>
</ul>
</div>

## Input & Output

So far, we have mostly been setting the values of variables used in our programs by hand and writing things to the terminal with the `print()` function.  We can use python to easily get data from a user, either form the command line, or from a file, and save our program output to a well-formatted file.

In [ ]:
# asking the user for input
N = input("Number of elements: ")
print(N)

In [ ]:
# all input is treating like a string
print(type(N))

In [ ]:
# we must manually convert if we need numerical data
Ni = int(N)
print(Ni)

Nf = float(N)
print(Nf)

N = float(input("enter a float: "))
print(type(N))

### Working with files
We can use the keyword `open` to open a file, but we must specify the mode, either `r` for read, `w` for write `r+` for read/write and `a` for append

In [ ]:
!cat 'data/in.dat'

In [ ]:
in_file = open('data/in.dat', 'r')

In [ ]:
# reading an entire file
print(in_file.read())

# we close when we are finished
in_file.close()

In [ ]:
# alternatively, we can open a file within a context
with open('data/in.dat', 'r') as f:
    read_data = f.read()
    print(read_data)
    
# the file is automatically closed when we leave the with context
print(f.closed)

In [ ]:
# we can get a list of all lines
with open('data/in.dat', 'r') as in_file:
    lines = in_file.readlines()

print(lines)

<div class="span alert alert-danger">
Note that the newline characters are included!
</div>

In [ ]:
# or just a single line
with open('data/in.dat', 'r') as in_file:
    line = in_file.readline()
print(line)

In [ ]:
# let's iterate through all the lines in the file
in_file = open('data/in.dat', 'r')
lines = in_file.readlines()
for line in lines:
    print (line)
in_file.close()

### Writing to files

In [ ]:
# specifiying 'w' will create a file if it doesn't exist and replace the content if it does
out_file = open('data/out.dat', 'w')

In [ ]:
# we can only write strings to files, so we must convert on input and output
for line in lines:
    a = int(line)
    b = a**2
    out_file.write('%d\n' % b)
out_file.close()   

In [ ]:
!cat data/out.dat

### Output formatting

Python supports various ways of formatting i/o. We have already seen the c-style `%` method which is my preferred method. There is a new-fangled way which is more flexible and
powerful,see: http://docs.python.org/tutorial/inputoutput.html use whichever you like for assignments.

A fully descriptive list of all formatting options can be found at https://docs.python.org/3/library/stdtypes.html#string-formatting

<div class="span alert alert-success">
<h2> Team programming challenge </h2>
<h3> Average energy for the simple harmonic oscillator at finite temperature</h3>
</div>

I have uploaded a data file on BlackBoard  called `sho_energy.dat`.  The line contains column headings with units in kelvin.

The next set of lines contain quantum Monte Carlo measurements for the kinetic and potential energy of the simple harmonic osscilator at $T = 0.5~\mathrm{K}$ where $\hbar \omega/k_{\mathrm{B}} = 1$.  The exact answer is known to be:
\begin{equation}
E(T) = \frac{\hbar \omega}{2} \coth \frac{\hbar \omega}{2 k_{\mathrm{B}} T}.
\end{equation}

Download the data file to your computer and write a program that loads the file from disk and stores it in a dictionary with labels taken from the column headers.  Compute the average total energy of all lines.  If you have extra time, compare with the exact result.